In [3]:
import os
import pandas as pd
import SimpleITK as sitk
import pickle
import numpy as np

image_dir = "data/images/trainval/pre-proc2"
landmarks_df = pd.read_pickle("data/landmarks_all.pkl")

In [6]:
class BinaryMask:

    def create_empty_image(self, image_ref):
        """Create an "empty" new image with the same dimensions and pixel type"""
        
        mask = sitk.Image(image_ref.GetSize(), sitk.sitkUInt8)
        mask.SetSpacing(image_ref.GetSpacing())
        mask.SetOrigin(image_ref.GetOrigin())
        mask.SetDirection(image_ref.GetDirection())

        print("---------Mask creation---------")
        print(f"Image size: {image_ref.GetSize()}")
        print(f"Mask size: {mask.GetSize()}")
        
        return mask

    def apply_landmarks(self, image, df, image_name):
        """Set voxel value to 1 at each landmark"""

        print("------Applying landmarks------")
        
        landmarks = df.columns.levels[0]
        
        for landmark in landmarks:
            x = round(df.loc[image_name, (landmark, 'X')])
            y = round(df.loc[image_name, (landmark, 'Y')])
            z = round(df.loc[image_name, (landmark, 'Z')])
            landmark_coords = (x, y, z)

            print(f"{landmark}: {landmark_coords}")
            
            # Set voxel value to 1
            image.SetPixel(landmark_coords, 1)  

        return image
    
    def binary_dilate(self, image):
        """Apply a Binary Dilation filter to make intensity 1 voxels become spheres"""

        dilate_filter = sitk.BinaryDilateImageFilter()
        dilate_filter.SetKernelRadius(7)
        dilate_filter.SetKernelType(sitk.sitkBall)
        dilate_filter.SetForegroundValue(1)
        
        dilated_image = dilate_filter.Execute(image)

        return dilated_image

    def execute(self, image, df, image_name):
        
        mask = self.create_empty_image(image)
        mask_annot = self.apply_landmarks(mask, df, image_name)
        mask_dilated = self.binary_dilate(mask_annot)
        
        return mask_dilated

In [10]:
image_names = sorted(os.listdir(image_dir))
for image_name in image_names:
    if image_name in landmarks_df.index:

        print(f"Image {image_name}:")
        print("--------------------------------")

        # Read image
        image = sitk.ReadImage(os.path.join(image_dir, image_name))

        # Create binary mask based on manually annotated landmarks
        mask = BinaryMask().execute(image, landmarks_df, image_name)

        sitk.WriteImage(mask, os.path.join(image_dir, "mask", image_name[:-7] + "_mask.nii.gz"))

        print("")

Image lleg_001_20100125b.nii.gz:
--------------------------------
---------Mask creation---------
Image size: (204, 251, 1040)
Mask size: (204, 251, 1040)
------Applying landmarks------
FKC: (149, 97, 412)
FME: (188, 92, 431)
FLE: (112, 96, 423)
FTP: (150, 122, 419)
TKC: (146, 86, 400)
FMCP: (171, 68, 419)
FLCP: (125, 72, 419)
TMCP: (181, 82, 391)
TLCP: (113, 82, 393)
FHC: (126, 121, 819)
FIP: (79, 110, 820)
TAC: (146, 76, 38)

Image lleg_003_20100305b.nii.gz:
--------------------------------
---------Mask creation---------
Image size: (226, 351, 1223)
Mask size: (226, 351, 1223)
------Applying landmarks------
FKC: (146, 113, 496)
FME: (185, 117, 506)
FLE: (104, 115, 498)
FTP: (143, 147, 494)
TKC: (142, 107, 470)
FMCP: (171, 89, 495)
FLCP: (118, 92, 495)
TMCP: (160, 88, 469)
TLCP: (128, 89, 465)
FHC: (145, 138, 870)
FIP: (93, 120, 873)
TAC: (160, 77, 139)

Image lleg_004_20100315a.nii.gz:
--------------------------------
---------Mask creation---------
Image size: (202, 324, 1239)
Mask